## 챌린지
---
(EN)
- Implement a complete RAG pipeline with a Stuff Documents chain.
- You must implement the chain manually.
- Give a ConversationBufferMemory to the chain.
- Use this document to perform RAG: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
- Ask the following questions to the chain:
    - Is Aaronson guilty?
    - What message did he write in the table?
    - Who is Julia?

(KR)
- Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
- 체인을 수동으로 구현해야 합니다.
- 체인에 ConversationBufferMemory를 부여합니다.
- 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
- 체인에 다음 질문을 합니다:
    - Aaronson 은 유죄인가요?
    - 그가 테이블에 어떤 메시지를 썼나요?
    - Julia 는 누구인가요?

In [9]:
from langchain_openai import ChatOpenAI
from langchain_unstructured import UnstructuredLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

# 캐시 저장 경로
cache_dir = LocalFileStore("./.cache/")

# Tiktoken: OpenAI에서 만든 tokenization library.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=950,
    chunk_overlap=100,
)

loader = UnstructuredLoader('./files/document.txt')

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

# 중복 요청 시 캐시된 결과를 반환
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# FAISS 라이브러리로 캐시에서 임베딩 벡터 검색
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# docs를 불러오는 역할
retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer, just say you don't know. Don't make it up:\n\n{context}"),
    ("human", "{question}")
])

# CASE: chain_type = 'stuff' (default)
# RunnablePassthrough: chain.invoke의 문장을 그대로 대입
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

def invoke_chain(question):
    response = chain.invoke(question)
    print(f"{response.content}\n\n")

In [10]:
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.




INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


He wrote in large clumsy capitals: "But then there came a sort of check."




INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Julia is a character mentioned in the provided text excerpts.


